<h3>Início</h3>

In [ ]:
from pandas_datareader import data as pd
from datetime import datetime

import yfinance as yf
yf.pdr_override()
inicio = "2014-09-17"
fim = "2023-03-14"
df = pd.DataReader("BTC-USD",inicio,fim)
df.index = df.index.date
df.index.name = "Date"

In [ ]:
print('Dataset BTC-USD\n')
df.head(10)

In [ ]:
df.info()

<h3>Coleta de Dados - Dados históricos do valor dos criptoativos</h3>

In [ ]:
from pandas_datareader import data as pdr
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import yfinance as yf
import pandas as pd

class CriptoClass(): pass
valores = CriptoClass()

yf.pdr_override()
criptoIndice         = 0
valores.listaCriptos = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'ADA-USD', 'SOL-USD'] 
valores.dataInicio   = ['2014-09-17','2017-11-09','2017-11-09','2017-11-09','2020-04-10']
valores.dataFim      = '2023-03-14'
dadosCripto = pdr.DataReader(valores.listaCriptos[criptoIndice], valores.dataInicio[criptoIndice], 
                             valores.dataFim)
dadosCripto.index = dadosCripto.index.tz_localize(None)

In [ ]:
dadosCripto.head()

In [ ]:
#Exibindo o resumo dos dados obtidos
print('Dataset BTC-USD\n')
print(dadosCripto.info())

In [ ]:
dadosCripto.index.duplicated().sum()

In [ ]:
(dadosCripto==0).astype(int).sum()

<h3>Coleta de Dados - Dados históricos do valor do Dólar</h3>

In [ ]:
dadosDolar = pd.read_csv("USD_BRL Dados Históricos (2).csv", encoding = "utf-8", parse_dates=True)
print('Dataset Dólar\n')
dadosDolar.head(10)

In [ ]:
dadosDolar.info()

<h3>Processamento dos dados - Dados históricos do valor dos criptoativos</h3>

In [ ]:
from pandas_datareader import data as pdr
yf.pdr_override()
criptoIndice         = 0
valores.listaCriptos = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'ADA-USD', 'SOL-USD'] 
#valores.dataInicio   = ['2014-09-17','2017-11-09'] #limite
#valores.dataInicio   = ['2017-09-01','2017-11-09'] #nova entrada
valores.dataInicio   = ['2017-09-01','2017-11-09','2017-11-09','2017-11-09','2020-04-10']
valores.dataFim      = '2023-03-14'
dadosCripto = pdr.DataReader(valores.listaCriptos[criptoIndice], valores.dataInicio[criptoIndice], valores.dataFim)
dadosCripto.index = dadosCripto.index.tz_localize(None)

In [ ]:
dadosCripto = dadosCripto.rename(columns={"Open": "CriptoAbertura"})
dadosCripto = dadosCripto.rename(columns={"High": "CriptoMaximo"})
dadosCripto = dadosCripto.rename(columns={"Low": "CriptoMinimo"})
dadosCripto = dadosCripto.rename(columns={"Close": "CriptoFechamento"})
dadosCripto = dadosCripto.rename(columns={"Adj Close": "CriptoFechamentoAjustado"})
dadosCripto = dadosCripto.rename(columns={"Volume": "CriptoVolume"})

In [ ]:
dadosCripto.head()

In [ ]:
dadosCripto.index.duplicated().sum()

In [ ]:
(dadosCripto==0).astype(int).sum()

In [ ]:
dadosCripto.corr()

In [ ]:
dadosCripto.corr()['CriptoFechamento']

In [ ]:
dadosCripto = dadosCripto.drop(columns = ["CriptoFechamentoAjustado"])
dadosCripto.head()

<h3>Processamento dos dados - Dados históricos do valor do dólar</h3>

In [ ]:
dadosDolar.index.duplicated().sum()

In [ ]:
dadosDolar.isnull().sum()

In [ ]:
dadosDolar[dadosDolar['Vol.'].isnull()].head(5)

In [ ]:
print("Porcentagem de dados nulos: %.2f%%" % 
      (dadosDolar['Vol.'].isnull().sum()/dadosDolar['Vol.'].isnull().count()*100))

In [ ]:
dadosDolar['Último'] = dadosDolar['Último'].apply(lambda x: str(x).replace(',','.')).astype(float)
dadosDolar['Abertura'] = dadosDolar['Abertura'].apply(lambda x: str(x).replace(',','.')).astype(float)
dadosDolar['Máxima'] = dadosDolar['Máxima'].apply(lambda x: str(x).replace(',','.')).astype(float)
dadosDolar['Mínima'] = dadosDolar['Mínima'].apply(lambda x: str(x).replace(',','.')).astype(float)
formato = "%d.%m.%Y"
dadosDolar['Data'] = dadosDolar['Data'].apply(lambda linha: datetime.strptime(linha, formato))
dadosDolar = dadosDolar.drop('Var%', axis=1)
dadosDolar = dadosDolar.drop('Vol.', axis=1)
dadosDolar = dadosDolar.rename(columns={"Último": "DolarFechamento",
                                   "Abertura": "DolarAbertura",
                                   "Máxima": "DolarMaximo",
                                   "Mínima": "DolarMinimo"})
dadosDolar = dadosDolar.set_index('Data')

In [ ]:
dadosDolar.head(5)

In [ ]:
dadosDolar[dadosDolar==0].count()

In [ ]:
dadosDolar.isnull().sum()

In [ ]:
dadosDolar.describe()

<h3>Integração dos datasets</h3>

In [ ]:
df = dadosCripto.join(dadosDolar)

In [ ]:
df.head(14)

In [ ]:
df.isnull().sum()

In [ ]:
df['DolarFechamento'] = df['DolarFechamento'].fillna(method='ffill')
df['DolarAbertura'] = df['DolarAbertura'].fillna(method='ffill')
df['DolarMaximo'] = df['DolarMaximo'].fillna(method='ffill')
df['DolarMinimo'] = df['DolarMinimo'].fillna(method='ffill')

In [ ]:
df.isnull().sum()

In [ ]:
df.head(20)

In [ ]:
df = df.assign(CriptoAberturaReal = df.CriptoAbertura * df.DolarAbertura)
df = df.assign(CriptoFechamentoReal = df.CriptoFechamento * df.DolarFechamento)
df = df.assign(CriptoMáximoReal = df.CriptoMaximo * df.DolarMaximo)
df = df.assign(CriptoMínimoReal = df.CriptoMinimo * df.DolarMinimo)
df = df.assign(CriptoVolumeReal = df.CriptoVolume * df.DolarFechamento)

df = df.rename(columns={"CriptoAberturaReal": "Cripto Abertura (R$)",
                        "CriptoFechamentoReal": "Cripto Fechamento (R$)",
                        "CriptoMáximoReal": "Cripto Máximo (R$)",
                        "CriptoMínimoReal": "Cripto Mínimo (R$)",
                        "CriptoVolumeReal": "Cripto Volume (R$)"})


In [ ]:
df.head()

In [ ]:
df = df.drop(columns = ["CriptoAbertura","CriptoMaximo","CriptoMinimo","CriptoFechamento",
                        "CriptoVolume","DolarFechamento","DolarAbertura","DolarMaximo","DolarMinimo"])

In [ ]:
df.isnull().sum()

In [ ]:
df.head(5)

<h3>Análise e Exploração dos Dados</h3>

In [ ]:
df.describe()

In [ ]:
df.boxplot(figsize=(14,6))

In [ ]:
df.drop('Cripto Volume (R$)', axis=1).boxplot(figsize=(14,6))

In [ ]:
df[['Cripto Abertura (R$)','Cripto Fechamento (R$)','Cripto Máximo (R$)',
                'Cripto Mínimo (R$)']].boxplot(figsize=(14,6))

In [ ]:
df[['Cripto Abertura (R$)','Cripto Fechamento (R$)','Cripto Máximo (R$)',
                'Cripto Mínimo (R$)']].plot.line(figsize=(17,6), linewidth=0.9 )

In [ ]:
df.hist(figsize = (15,10))
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(), vmin=-1.0, vmax=1.0, annot=True, cmap='coolwarm', linewidths=0.1)
plt.show()

In [ ]:
from scipy.stats import norm

# plotando a variação de preço do ativo por dia
fig, ax = plt.subplots(figsize=(12,5))
df['Cripto Fechamento (R$)'].plot(ax=ax)
ax.set_title('Preço diário Bitcoin')
ax.set_xlabel('')
ax.set_ylabel('R$')
ax.tick_params(rotation=0)

# plotando as distribuições dos preços, por DIA
plt.figure(figsize=(8,5))
ax = sns.distplot(df['Cripto Fechamento (R$)'], fit=norm, kde=True)
ax.set_xlabel('Cripto Fechamento (R$)')
ax.set_title('Distribuição dos PREÇOS de Bitcoin Diário')

In [ ]:
# verificar a necessidade de retirar a coluna Volume

In [ ]:
dfNovo = pd.DataFrame()
dfNovo = df.drop(['Cripto Abertura (R$)','Cripto Máximo (R$)','Cripto Mínimo (R$)'], axis=1)

In [ ]:
dfNovo.head()

<h3>Análise exploratória de dados e Featuring Engineering para modelo ARIMA</h3>

<h3>EDA para ARIMA</h3>
<h4>Os principais pontos para utilizar o modelo ARIMA são:</h4>
</br>
<li> Sazonalidade </li>
<li> Série estacionária — definição do parâmetro d </li>
<li> Média Móvel </li>
<li> Outliers </li>
<li> Função de Autocorrelação Parcial (p) </li>
<li> Função de Autocorrelação (q) </li>

<h3>Identificação da Sazonalidade</h3>

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# decompondo a série temporal em seus componentes de tendência, sazonalidade e erro
decomposicao = seasonal_decompose(dfNovo['Cripto Fechamento (R$)'], model='additive', period=90)

# plotando o gráfico para cada componente
plt.figure(figsize=(15,10))
plt.subplot(411)
plt.plot(dfNovo['Cripto Fechamento (R$)'], label='Original')
plt.legend(loc='upper left')
plt.subplot(412)

plt.plot(decomposicao.trend, label='Tendência')
plt.legend(loc='upper left')
plt.subplot(413)

plt.plot(decomposicao.seasonal, label='Sazonalidade')
plt.legend(loc='upper left')
plt.subplot(414)

plt.plot(decomposicao.resid, label='Resíduos')
plt.legend(loc='upper left')
plt.tight_layout()


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Teste de Dickey-Fuller
resultado = adfuller(dfNovo['Cripto Fechamento (R$)'])

# Imprimindo o resultado do teste
print('Estatística do teste: {:.2f}'.format(resultado[0]))
print('Valor-p: {:.2f}'.format(resultado[1]))
print('Valores críticos:')
for chave, valor in resultado[4].items():
    print('\t{}: {:.2f}'.format(chave, valor))

In [ ]:
# Diferenciação de primeira ordem
df_arima = dfNovo.copy()
df_arima['retorno_percentual'] = dfNovo['Cripto Fechamento (R$)'].pct_change()
df_arima.dropna(inplace=True)

# Plot da série resultante e sua distribuição
fig, axs = plt.subplots(1,2,figsize=(15,5))
axs[0].plot(df_arima['retorno_percentual'])
axs[0].set_title('Série resultante após diferenciação de primeira ordem')
ax2 = sns.distplot(df_arima['retorno_percentual'], fit=norm, kde=True, ax=axs[1])
ax2.set_title('Distribuição de probabilidade da série resultante')


<h3>Verificação da estacionariedade da série - definição do parâmetro d</h3>

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Teste de Dickey-Fuller
resultado = adfuller(df_arima['retorno_percentual'])

# Imprimindo o resultado do teste
print('Estatística do teste: {:.2f}'.format(resultado[0]))
print('Valor-p: {:.2f}'.format(resultado[1]))
print('Valores críticos:')
for chave, valor in resultado[4].items():
    print('\t{}: {:.2f}'.format(chave, valor))

In [ ]:
df_arima.head()

<h3>Identificação de outliers</h3>

In [ ]:
# Cálculo da média móvel e do desvio padrão móvel
rolling_mean = df_arima['retorno_percentual'].rolling(window=30).mean()
rolling_std = df_arima['retorno_percentual'].rolling(window=30).std()

# Cálculo dos limites de outlier
upper_limit = rolling_mean + 3 * rolling_std
lower_limit = rolling_mean - 3 * rolling_std

# Marcação dos outliers
outliers = (df_arima['retorno_percentual'] > upper_limit) | (df_arima['retorno_percentual'] < lower_limit)
outliers_idx = df_arima.index[outliers]

# Plot da série resultante com limites de outlier
plt.figure(figsize=(15,5))
plt.plot(df_arima['retorno_percentual'], label='Cripto Fechamento (R$) - 1ª Diferenciação')
plt.scatter(outliers_idx, df_arima['retorno_percentual'][outliers_idx], color='red', label='Outliers')
plt.title('Identificação de Outliers')
plt.legend(loc='lower left')
plt.show()


In [ ]:
from scipy.stats import mstats

# Cálculo da média móvel e do desvio padrão móvel
rolling_mean = df_arima['retorno_percentual'].rolling(window=30).mean()
rolling_std = df_arima['retorno_percentual'].rolling(window=30).std()

# Cálculo dos limites de outlier
upper_limit = rolling_mean + 3 * rolling_std
lower_limit = rolling_mean - 3 * rolling_std

# Marcação dos outliers
outliers = (df_arima['retorno_percentual'] > upper_limit) | (df_arima['retorno_percentual'] < lower_limit)
outliers_idx = df_arima.index[outliers]

# Winsorização dos dados
winsorized = mstats.winsorize(df_arima['retorno_percentual'], limits=[0.05, 0.05])

# Substituição dos outliers pelos valores winsorizados
df_arima['retorno_winsorized'] = np.where(outliers, winsorized, df_arima['retorno_percentual'])

# Plot da série resultante
plt.figure(figsize=(15,5))
plt.plot(df_arima['retorno_winsorized'], label='Cripto Fechamento (R$) - 1ª Diferenciação')
plt.scatter(outliers_idx, df_arima['retorno_percentual'][outliers_idx], color='red', label='Outliers')
plt.title('Tratamento de Outliers com Winsorize')
plt.legend(loc='lower left')
plt.show()


<h3>Análise da função de Autocorrelação Parcial - definição do parâmetro (p)</h3>

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

fig, ax = plt.subplots(figsize=(15, 8))

# Plot da PACF
plot_pacf(df_arima['retorno_winsorized'].dropna(), lags=30, ax=ax, method='ywm')
ax.set_title('Autocorrelação Parcial PACF - Bitcoin')
ax.set_ylim(bottom=-0.08, top=1.1)
plt.show()


<h3>Análise da função de Autocorrelação - definição do parâmetro (q)</h3>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

fig, ax = plt.subplots(figsize=(15, 8))

# Plot da ACF
plot_acf(df_arima['retorno_winsorized'].dropna(), lags=30, ax=ax)
ax.set_title('Autocorrelação ACF - Bitcoin')
ax.set_ylim(bottom=-0.08, top=1.1)
plt.show()

In [ ]:
df_arima.tail()

In [ ]:
df.head(5)

In [ ]:
dfNovo.head(5)

<h3>Análise exploratória de dados e Feature Engineering para outros modelos</h3>

<h3>Média Móvel</h3>

In [ ]:
def media_movel(df, nome, coluna, dias):
    df[nome] = df[coluna].rolling(dias).mean()
    return df

In [ ]:
dfNovo = media_movel(dfNovo,'MediaMovel_7', ['Cripto Fechamento (R$)'], 7)
dfNovo = media_movel(dfNovo,'MediaMovel_9', ['Cripto Fechamento (R$)'], 9)
dfNovo = media_movel(dfNovo,'MediaMovel_21', ['Cripto Fechamento (R$)'], 21)

In [ ]:
dfNovo.dropna(inplace=True)
dfNovo.head()

In [ ]:
# Plotar o gráfico com as médias móveis
media = pd.DataFrame(dfNovo)
media = media.tail(90)

plt.figure(figsize=(20, 10))
plt.plot(media.index, media['Cripto Fechamento (R$)'], label='Preço', linewidth=4.5)
plt.plot(media.index, media['MediaMovel_7'], label='Média Móvel 7')
plt.plot(media.index, media['MediaMovel_9'], label='Média Móvel 9')
plt.plot(media.index, media['MediaMovel_21'], label='Média Móvel 21')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.title('Médias Móveis de 7, 9 e 21 dias')
plt.legend()
plt.grid(True)
plt.show()

<h3>Índice de Força Relativa (IFR)</h3>

In [ ]:
import talib

#Indicador IFR (Indice de Força Relativa)

IFR_PERIODO = 14 # definindo o período considerado para cálculo de IFR
dfNovo['IFR'] = talib.RSI(dfNovo['Cripto Fechamento (R$)'], IFR_PERIODO)  # criando a feature IFR
# IFR < 30 -> 0 (comprar)
# IFR > 70 -> 1 (vender)
# Else -> 2 (nada)
dfNovo['IFR - Indicador'] = dfNovo['IFR'].apply(lambda x: 0 if x<30 else 1 if x>70 else 2) 
display(dfNovo[['Cripto Fechamento (R$)','IFR','IFR - Indicador']].tail(5))

In [ ]:
plt.title("Variação IFR Bitcoin - Indicação da Faixa de compra e venda")
dfNovo['IFR'].plot()
plt.axhline(y=30, color='black', linestyle='--')
plt.axhline(y=70, color='black', linestyle='--')
plt.axhspan(30, 70, color='thistle')
plt.ylim(0, 100)

In [ ]:
# Análise do indice de força relativa
print(dfNovo["IFR - Indicador"].value_counts())
print(dfNovo["IFR - Indicador"].value_counts(normalize=True).map("{:.1%}".format))

<h3>Bandas de Bollinger (BB)</h3>

In [ ]:
#Indicador Bandas de Bollinger

# calculando a média móvel e limites superior e inferior
# limites com base em 2 desvios padrão 
superior, media, inferior = talib.BBANDS(dfNovo['Cripto Fechamento (R$)'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
# criando features para a média e os limites
dfNovo['Banda Superior'] = superior
dfNovo['Banda Média'] = media

dfNovo['Banda Inferior'] = inferior
dfNovo['BB'] = (dfNovo['Cripto Fechamento (R$)'] - dfNovo['Banda Inferior'])/(dfNovo['Banda Superior'] - 
                                                                              dfNovo['Banda Inferior'])
dfNovo.dropna(inplace=True)
# BB < 0 -> 0 (comprar)
# BB > 1 -> 1 (vender)
# Else -> 2 (nada)
dfNovo['BB - Indicador'] = dfNovo['BB'].apply(lambda x: 0 if x<0 else 1 if x>1 else 2) 
display(dfNovo[['Cripto Fechamento (R$)','BB','BB - Indicador']].tail(5))

In [ ]:
plt.title("Variação BB Bitcoin - Indicação da Faixa de compra e venda")
dfNovo['BB'].plot()
plt.axhline(y=0, color='black', linestyle='--')
plt.axhline(y=1, color='black', linestyle='--')
plt.axhspan(0, 1, color='thistle')
plt.ylim(-1, 2)

In [ ]:
# Análise da bandas de bollinger
print(dfNovo["BB - Indicador"].value_counts())
print(dfNovo["BB - Indicador"].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
dfNovo.head()

In [ ]:
# Plot the upper, mid and lower bands with prices
import plotly.graph_objs as go
fig = go.Figure(data=[go.Scatter(
    name="Preço",
    x=dfNovo.index,
    y=dfNovo["Cripto Fechamento (R$)"],
    mode='lines'),
    go.Scatter(name="Banda Superior", x=dfNovo.index, y=dfNovo['Banda Superior'], line=dict(color='tomato', width=2)),
    go.Scatter(name="Banda Média", x=dfNovo.index, y=dfNovo['Banda Média'], line=dict(color='green', width=2)),
    go.Scatter(name="Banda Inferior", x=dfNovo.index, y=dfNovo['Banda Inferior'], line=dict(color='tomato', width=2))])


# Customise and show plot
fig.update_layout(title='Preços BITCOIN ')
fig['layout']['yaxis']['title']='Preços'
fig.show()

In [ ]:
dfNovo.head()

<h3>Suporte e resistência</h3>

In [ ]:
# função de suporte
def func_suporte(df,i):
  suporte = (df['Cripto Mínimo (R$)'][i] < df['Cripto Mínimo (R$)'][i-1]
             and df['Cripto Mínimo (R$)'][i] < df['Cripto Mínimo (R$)'][i+1]
             and df['Cripto Mínimo (R$)'][i+1] < df['Cripto Mínimo (R$)'][i+2]
             and df['Cripto Mínimo (R$)'][i-1] < df['Cripto Mínimo (R$)'][i-2])
  return suporte

# função de resistencia
def func_resistencia(df,i):
  resistencia = (df['Cripto Máximo (R$)'][i] > df['Cripto Máximo (R$)'][i-1]
                and df['Cripto Máximo (R$)'][i] > df['Cripto Máximo (R$)'][i+1]
                and df['Cripto Máximo (R$)'][i+1] > df['Cripto Máximo (R$)'][i+2]
                and df['Cripto Máximo (R$)'][i-1] > df['Cripto Máximo (R$)'][i-2])
  return resistencia


# suporte -> 0 (comprar)
# resistência -> 1 (vender)
# outros (2)

# criando feature com valores 2
dfNovo['Suporte/Resistencia'] = 2

# definindo os valores 1 e 0
for i in range(2, dfNovo.shape[0] - 2):
    if func_suporte(df,i):
        dfNovo['Suporte/Resistencia'][i] = 0 # definindo 0 para suporte
    elif func_resistencia(df,i):
        dfNovo['Suporte/Resistencia'][i] = 1 # definindo 1 para resistência

In [ ]:
# Análise das regiões de suporte e resistência
print(dfNovo['Suporte/Resistencia'].value_counts())
print(dfNovo['Suporte/Resistencia'].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
dfNovo.head()

<h3>Linhas de tendência</h3>

In [ ]:
# linhas de tendência
dfNovo2 = dfNovo.reset_index()
dfNovo['Correlacao'] = (dfNovo2['Cripto Fechamento (R$)'].rolling(20).corr(pd.Series(dfNovo2.index))).tolist()
dfNovo.dropna(inplace=True)

def func_definicao(x):
    if x<=-0.5:
        return -1
    elif x>-0.5 and x<0.5:
        return 0
    else:
        return 1
    
# tendência de alta -> 1 (comprar)
# tendência de baixa -> -1 (vender)
# fase de lateralidade (0)
    
dfNovo['LTA/LTB - Indicador'] = dfNovo['Correlacao'].apply(func_definicao)

# gráficos
sns.scatterplot(x = dfNovo.index, y = dfNovo['Correlacao'])
sns.countplot(x = dfNovo['LTA/LTB - Indicador'])

In [ ]:
dfNovo.head()

In [ ]:
df_arima['Cripto Fechamento (R$)'].tail()

<h1>Criação de Modelos de Machine Learning<h1/>

<h3>Modelo ARIMA<h3/>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [ ]:
# Separar os dados de treino e os dados de teste
treino = df_arima['Cripto Fechamento (R$)'].iloc[:-60]
teste = df_arima['Cripto Fechamento (R$)'].iloc[-60:]

plt.title('Dados de treino e Dados de teste',size=15)
treino.plot(figsize=(15,6))
teste.plot()
plt.xlabel('Série',size=15)
plt.legend(['Treino','Teste']);

In [ ]:
# Criar modelo ARIMA
model = ARIMA(treino, order=(1, 1, 1))
model_fit = model.fit()

# Realizar previsão
step = len(teste)
previsao = model_fit.forecast(step)
previsao = pd.Series(previsao, index = teste[:step].index)

# Plotar resultados
plt.figure(figsize=(16, 8))
plt.plot(treino[:], label='Treino')
plt.plot(teste[:step], label='Teste')
plt.plot(previsao, label='Previsto')
plt.title('Previsto vs atual')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [ ]:
# Calcular as métricas
mae = mean_absolute_error(teste, previsao)
rmse = mean_squared_error(teste, previsao, squared=False)


# Imprimir as métricas
print("O MAE do modelo ARIMA(1,1,1):", mae)
print("O RMSE do modelo ARIMA(1,1,1):", rmse)

In [ ]:
print("O R² do modelo ARIMA(1,1,1) é:", r2_score(teste, previsao))

In [ ]:
# Criar um dataframe para o preço
precos = df_arima['Cripto Fechamento (R$)']

# Em seguida, vamos atribuir 70% dos dados para treinamento e 30% para teste.
X = precos.values
size = int(len(X) * 0.7)
treino_walk, teste_walk = X[0:size], X[size:len(X)]
historico = [x for x in treino_walk]
previsoes_walk = list()

In [ ]:
# Abordagem Walk-Forward 
for i in range(len(teste_walk)):
    model = ARIMA(historico, order=(1,1,1))
    model_fit = model.fit()
    saida = model_fit.forecast()
    previsto = saida[0]
    previsoes_walk.append(previsto)
    esperado = teste_walk[i]
    historico.append(esperado)
    print('Previsto=%f, Esperado=%f' % (previsto, esperado))

In [ ]:
mae_walk = mean_absolute_error(teste_walk, previsoes_walk)
rmse_walk = mean_squared_error(teste_walk, previsoes_walk, squared=False)

# Imprimir as métricas
print("O MAE do modelo ARIMA(1,1,1):", mae_walk)
print("O RMSE do modelo ARIMA(1,1,1):", rmse_walk)

# Plotar previsões em relação aos resultados reais
plt.figure(figsize=(15,8))
plt.plot(teste_walk, label='Preço Atual')
plt.plot(previsoes_walk, label='Previsões', color='red')
plt.xlabel('Dias')
plt.ylabel('R$ Preços')
plt.title('Previsto vs. Preço Atual BTC')
plt.legend(loc='best')
plt.show()

In [ ]:
mae_walk = mean_absolute_error(teste_walk, previsoes_walk)
rmse_walk = mean_squared_error(teste_walk, previsoes_walk, squared=False)

# Imprimir as métricas
print("O MAE do modelo ARIMA(1,1,1):", mae_walk)
print("O RMSE do modelo ARIMA(1,1,1):", rmse_walk)

# Plot de 60 dias para ver com mais precisão como o modelo funciona com seu atraso.
plt.plot(teste_walk[-60:], label='Preço Atual')
plt.plot(previsoes_walk[-60:], label='Previsto', color='red')
plt.xlabel('Dias')
plt.ylabel('R$ Preços')
plt.title('Previsto vs. Preço Atual BTC')
plt.legend(loc='best')
plt.show()

In [ ]:
print("O R² do modelo ARIMA(1,1,1) é:", r2_score(teste_walk, previsoes_walk))

<h3>Modelos de Regressão<h3/>

In [ ]:
df_criptoFecha = pd.DataFrame()
df_criptoFecha = df_criptoFecha.assign(CriptoFechaReal = dfNovo['Cripto Fechamento (R$)'].copy())
df_criptoFecha = df_criptoFecha.rename(columns={"CriptoFechaReal": "Cripto Fechamento (R$)"})

In [ ]:
# Cria função que defasa os retornos diários (lags)
def constroi_lags(dataset, colunas, qtd_lags):
    dataset_lag = dataset.copy()
    for col in colunas:
        for i in range(1, qtd_lags + 1):
            lag_column_name = f"{col}_lag_{i}"
            dataset_lag[lag_column_name] = dataset_lag[col].shift(i)
    dataset_lag.dropna(inplace=True)
    return dataset_lag

In [ ]:
df_criptoFecha = constroi_lags(df_criptoFecha, ['Cripto Fechamento (R$)'], 50)
df_criptoFecha.tail()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# Remove linhas com valores ausentes
df_criptoFecha = df_criptoFecha.dropna()

# Converter colunas para tipo float
df_criptoFecha = df_criptoFecha.astype(float)

# Separar variáveis dependentes e independentes
y = df_criptoFecha['Cripto Fechamento (R$)']
X = df_criptoFecha.drop(['Cripto Fechamento (R$)'], axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27)

In [ ]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

modelos_convencionais = {
    'Árvore': DecisionTreeRegressor(max_depth=8),  
    'RandomForest': RandomForestRegressor(max_depth=8),  
    'LGBM': LGBMRegressor(max_depth=8) 
}

# Dicionário para armazenar as avaliações de cada modelo
avaliacao = {}  

for nome, model in modelos_convencionais.items():
    # Avalia a crossvalidação
    scores = cross_val_score(
        model, X_train, y_train, cv=TimeSeriesSplit(n_splits=5), scoring='neg_mean_absolute_error', n_jobs=8
    )
    avg_score = -scores.mean()  # Calcula a média dos scores e inverte o sinal para obter um valor positivo
    avaliacao[nome] = avg_score  # Armazena o resultado da avaliação no dicionário

avaliacao  # Retorna o dicionário com as avaliações de cada modelo


In [ ]:
import matplotlib.pyplot as plt

# Dados
dados = avaliacao

# Criar as barras horizontais
plt.figure(figsize=(16, 6))
plt.barh(range(len(dados)), list(dados.values()))

# Adicionar os rótulos do eixo y
plt.yticks(range(len(dados)), list(dados.keys()))

# Definir o título do gráfico
plt.title('Comparação de Modelos')

# Exibir o gráfico
plt.show()


In [ ]:
# maeRandomForest = {10: 3687.509308532358, 
#                    15: 3679.4508023720323, 
#                    20: 3500.647630313722, 
#                    25: 3650.942082980467, 
#                    30: 3402.0801850812522, 
#                    40: 3452.729961408718, 
#                    50: 3638.2137870798892, 
#                    100: 3711.6300583976567, 
#                    150: 3516.341485960473, 
#                    200: 3683.8362375304023, 
#                    250: 3823.9704126606062}

In [ ]:
import matplotlib.pyplot as plt

# Dados
maeRandomForest = {  10: 3687.509308532358, 15: 3679.4508023720323, 20: 3500.647630313722, 25: 3650.942082980467, 30: 3402.0801850812522, 
           40: 3452.729961408718, 50: 3638.2137870798892, 100: 3711.6300583976567, 150: 3516.341485960473, 
           200: 3683.8362375304023, 250: 3823.9704126606062}

plt.figure(figsize=(16, 8))

# Obter os valores das chaves e dos valores
x = list(maeRandomForest.keys())
y = list(maeRandomForest.values())

# Criar o gráfico de linhas
plt.plot(x, y)

# Definir os rótulos dos eixos
plt.xlabel('Dias de lags (defasagem)')
plt.ylabel('Valor do MAE (menor valor é preferível)')

# Definir o título do gráfico
plt.title('Evolução do MAE à medida que aumentamos o número de lags para o treinamento')

# Exibir o gráfico
plt.show()


<h3>Coluna Target<h3/>

In [ ]:
dfNovo.head()

In [ ]:
pd.set_option('display.max_columns', 11)
pd.set_option('display.max_rows', 11)

In [ ]:
dfClassif = pd.DataFrame()
dfClassif = dfNovo.copy()

In [ ]:
dfClassif = constroi_lags(dfClassif, ['Cripto Fechamento (R$)'], 30)
dfClassif.head()

In [ ]:
# valor_antigo = 100
# valor_novo = 95
# diferenca = valor_novo - valor_antigo
# diferenca_percentual = (diferenca / valor_antigo) * 100
# diferenca_percentual

In [ ]:
def target(df):
    df['lag_1'] = dfClassif['Cripto Fechamento (R$)'].shift(1)
    df['dif'] = dfClassif['Cripto Fechamento (R$)'] - df['lag_1']
    
    # criando o Target para COMPRA (diferença positiva), em relação ao dia anterior.
    df['Target'] = df['dif'].apply(lambda x: 1 if x > 0 else 0 )
    
    return df

In [ ]:
# def target(df):
#     df['lag_1'] = dfClassif['Cripto Fechamento (R$)'].shift(1)
#     df['dif'] = dfClassif['Cripto Fechamento (R$)'] - df['lag_1']
#     df['perc'] = (df['dif'] / df['lag_1'] ) * 100
#     # criando a target de subida ou descida do valor de fechamento
#     # valor caiu  -> 0  (diferença negativa)
#     # valor subiu -> 1  (diferença positva)
#     df['Target'] = df['perc'].apply(lambda x: 1 if x >= 1 else 0 )
#     return df

In [ ]:
# target(dfClassif)
# dfClassif = dfClassif.drop(['dif','lag_1', 'IFR - Indicador','BB - Indicador','Suporte/Resistencia','LTA/LTB - Indicador'], axis=1)

In [ ]:
target(dfClassif)
#dfClassif = dfClassif.drop(['dif','lag_1', 'perc'], axis=1)
dfClassif = dfClassif.drop(['dif','lag_1'], axis=1)

In [ ]:
dfClassif.head(10)

In [ ]:
# # Verificar o balanceamento da Target no dataset

# const = pd.DataFrame(columns=['metric','0','1'])
# const = const.append({'metric':'Target','0':dfClassif['Target'][dfClassif['Target']==0].count(),'1':dfClassif['Target'][dfClassif['Target']==1].count()}, ignore_index=True)
# const_porcent = np.sum(const[const.metric=='Target'][['1']].values)/np.sum(const[const.metric=='Target'][['0','1']].values)
# print(const)
# print('Frequência targetH: %.2f%%' % (100*const_porcent))

In [ ]:
cont = dfClassif['Target'].value_counts()
percent = cont / len(dfClassif) * 100
result = pd.concat([cont, percent.map("{:.2f}".format)], axis=1, keys=['Quant.', '%'])
print(result)

<h3>Modelos de Classificação<h3/>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, auc


dfClassif = dfClassif.dropna()
for coluna in dfClassif.columns: dfClassif[coluna] = dfClassif[coluna].astype(float)
y = dfClassif['Target']
X = dfClassif.drop(['Target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Dataset utilizando a útltima linha, para realizar a previsão do Target.
X_pred = dfClassif.drop(['Target'], axis=1).tail(1)
for coluna in X_pred.columns: X_pred[coluna] = X_pred[coluna].astype(float)

In [ ]:
resultados = pd.DataFrame(columns=('Cripto','Modelo','Precision_Teste','Recall_Teste',
                                  'Precision_Completo', 'Recall_Completo',
                                   'Previsão'))
resultados['Modelo'] = ['Árvore de Decisão','Random Forest','XGBoost']
resultados['Cripto'] = valores.listaCriptos[criptoIndice][:3]

<h5>Árvore de Decisão<h5/>


In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=3)
model = model.fit(X_train, y_train)
y_prev = model.predict(X_test)
accuracy = accuracy_score(y_test, y_prev)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# Criando a matriz de confusão
ax = sns.heatmap(confusion_matrix(y_test, model.predict(X_test)),
                 annot=True,
                 annot_kws={"fontsize":10},
                 fmt = 'd',
                 cmap = 'Blues')

In [ ]:
# Criando as classificações de reporte
print('\n- Reporte completo')
print(classification_report(y, model.predict(X)))
print('\n- Reporte teste')
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
resultados.loc[resultados['Modelo'] == 'Árvore de Decisão', 'Precision_Teste'] = precision_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'Árvore de Decisão', 'Recall_Teste'] = recall_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'Árvore de Decisão', 'Precision_Completo'] = precision_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'Árvore de Decisão', 'Recall_Completo'] = recall_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'Árvore de Decisão', 'Previsão'] = model.predict(X_pred).astype(float)
resultados

<h5>Random Forest<h5/>


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=3)
model = model.fit(X_train, y_train)
y_prev = model.predict(X_test)
accuracy = accuracy_score(y_test, y_prev)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# Criando a matriz de confusão
ax = sns.heatmap(confusion_matrix(y_test, model.predict(X_test)),
                 annot=True,
                 annot_kws={"fontsize":10},
                 fmt = 'd',
                 cmap = 'Blues')

In [ ]:
# Criando as classificações de reporte
print('\n- Reporte completo')
print(classification_report(y, model.predict(X)))
print('\n- Reporte teste')
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
resultados.loc[resultados['Modelo'] == 'Random Forest', 'Precision_Teste'] = precision_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'Random Forest', 'Recall_Teste'] = recall_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'Random Forest', 'Precision_Completo'] = precision_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'Random Forest', 'Recall_Completo'] = recall_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'Random Forest', 'Previsão'] = model.predict(X_pred).astype(float)
resultados

<h5>XGBoost<h5/>


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=3, objective='binary:logitraw')
model.fit(X_train, y_train)
y_prev = model.predict(X_test)
accuracy = accuracy_score(y_test, y_prev)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# criando a matriz de confusão
ax = sns.heatmap(confusion_matrix(y_test, model.predict(X_test)),
                 annot=True,
                 annot_kws={"fontsize":10},
                 fmt = 'd',
                 cmap = 'Blues')

In [ ]:
# Criando as classificações de reporte
print('\n- Reporte completo')
print(classification_report(y, model.predict(X)))
print('\n- Reporte teste')
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
resultados.loc[resultados['Modelo'] == 'XGBoost', 'Precision_Teste'] = precision_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'XGBoost', 'Recall_Teste'] = recall_score(y_test, model.predict(X_test))
resultados.loc[resultados['Modelo'] == 'XGBoost', 'Precision_Completo'] = precision_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'XGBoost', 'Recall_Completo'] = recall_score(y, model.predict(X))
resultados.loc[resultados['Modelo'] == 'XGBoost', 'Previsão'] = model.predict(X_pred).astype(float)
resultados

In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(20, 20))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Feature Importance')